In [ ]:
# !pip install cartoframes==v1.0b7

In [ ]:
import cartoframes as cf
from cartoframes import CartoDataFrame 
from cartoframes.auth import Credentials, set_default_credentials, get_default_credentials
from cartoframes.viz import Map, Layer, basemaps
from cartoframes.viz.helpers import *
from cartoframes.data.services import Isolines

from cartoframes.data.observatory import Dataset
from cartoframes.data.observatory import Variable
from cartoframes.data.observatory import Enrichment
from cartoframes.data.observatory import Catalog


import ipywidgets as widgets
from ipywidgets import Output, Tab
from IPython.display import clear_output

import pandas as pd

cf.__version__

In [ ]:
def gpd_bbox(bounds):
    '''
    bounds (list): min_lng, min_lat, max_lng, max_lat
    '''
    from shapely.geometry import Polygon
    import geopandas as gpd
    import pandas as pd
    bbox = gpd.GeoDataFrame(pd.DataFrame([Polygon.from_bounds(*bounds)], columns=['geometry']))
    return bbox

def tile_cover(geometry, z):
    import geopandas as gpd
    from supermercado import burntiles, super_utils
    from pygeotile.tile import Tile
    geo = gpd.GeoSeries([geometry]).__geo_interface__['features'][0]
    geo = [f for f in super_utils.filter_polygons([geo])]
    return [Tile.from_google(*geo).quad_tree for geo in [f for f in burntiles.burn(geo, z)]]

def quad_to_poly(quad):
    from shapely.geometry import Polygon
    from pygeotile.tile import Tile
    from pygeotile import tile
    bbox = tile.Tile.from_quad_tree(quad).bounds
    # min_lng, min_lat, max_lng, max_lat
    poly = Polygon.from_bounds(bbox[0][1], bbox[0][0], bbox[1][1], bbox[1][0]) 
    return poly

def gpd_grid(geometry, z, tag=None):
    '''
    draw grid and return gpd.GeoDataFrame()
    '''
    from shapely.geometry import Polygon, MultiPolygon
    import pandas as pd
    import geopandas as gpd
    
    if isinstance(geometry, Polygon) or isinstance(geometry, MultiPolygon):
        qt_list = tile_cover(geometry, z)
    elif isinstance(geometry, list) and len(geometry)==4:
        qt_list = tile_cover(gpd_bbox(geometry).geometry.values[0], z)
    else:
        raise TypeError('''
        geometry should be either 
        shapely.Polygon (or shapely.MultiPolygon) 
        or list (min_lng, min_lat, max_lng, max_lat)')
        ''')
    grid = pd.DataFrame(qt_list, columns=['qt'])
    grid['geometry'] = grid['qt'].apply(lambda x: quad_to_poly(x))
    grid['tag'] = [tag] * len(grid)
    return gpd.GeoDataFrame(grid)

def qt_convert(lat, lng, zoom=19):
    '''
    lat, lng, zoom=18(default)
    '''
    from pygeotile.tile import Tile
    try:
        return Tile.for_latitude_longitude(lat, lng, zoom=zoom).quad_tree
    except:
        pass
    
# def run_api(ngrok_address, api_call="add/1/2", pause=1, timeout=600):
#     import time
#     import requests
#     exec_call = ngrok_address + api_call
#     cid = requests.get(exec_call).text.split('>')[1].split('<')[0].split('check status of')[1].strip()
#     output_call = ngrok_address + f"check/{cid}?external=True"
    
#     result = requests.get(output_call)
#     start_time = time.time()
#     process_time = float('inf')
#     while result.text=='PENDING' :
#         time.sleep(pause)
#         result = requests.get(output_call)
#         process_time = time.time() - start_time
#         print(process_time)
#         if process_time>timeout:
#             break
#     return result

def run_api(ngrok_address, api_call="add/1/2", pause=1, timeout=600):
    import time
    import requests
    exec_call = ngrok_address + api_call
    cid = requests.get(exec_call).text.split('>')[1].split('<')[0].split('check status of')[1].strip()
    output_call = ngrok_address + f"check/{cid}?external=True"
    
    result = requests.get(output_call)
    start_time = time.time()
    process_time = float('inf')
    while result.text=='PENDING' :
        time.sleep(pause)
        result = requests.get(output_call)
        process_time = time.time() - start_time
        print(process_time)
        if process_time>timeout:
            break
    return result


In [ ]:
auth_username = widgets.Text(
                    placeholder='Username',
                    description='Username:',
                    disabled=False)
auth_apikey = widgets.Text(
                    placeholder='API Key',
                    description='API Key:',
                    disabled=False)
auth_confirm = widgets.Button(description="Authentication",
                              layout=widgets.Layout(width='25%'))
auth_confirm.style.button_color = 'lightgreen'

@auth_confirm.on_click
def auth(*ignore):
    auth_apikey.disabled=True
    auth_username.disabled=True
    set_default_credentials(username=auth_username.value,
                            api_key=auth_apikey.value)
    

################################################################################################################################

data_tablename = widgets.Text(
                    placeholder='Table Name',
                    description='Table Name:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))

data_confirm = widgets.Button(description="Confirm",
                              layout=widgets.Layout(width='25%'))
data_confirm.style.button_color = 'lightgreen'

data_preview = widgets.Button(description="Preview",
                              layout=widgets.Layout(width='25%'))
data_preview.style.button_color = 'lightblue'
data_preview_map = widgets.HTML()

@data_preview.on_click
def preview(*ignore):
    preview_map = Map(Layer(data_tablename.value), show_info=True)
    data_preview_map.value = preview_map._repr_html_()
    
@data_confirm.on_click
def tablename_confirm(*ignore):
    data_tablename.disabled=True
    
################################################################################################################################

arrow = widgets.HTML("""<font size="+1000">&#10144;</font>""")

enrich_step_origin = widgets.ToggleButton(
    value=True,
    description='Origin',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

enrich_step_isochrone = widgets.ToggleButton(
    value=True,
    description='Isochrone',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)


enrich_step_run = widgets.Button(
    value=True,
    description='Enrich',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

################################################################################################################################

@enrich_step_origin.observe
def step_origin_check(*ignore):
    if enrich_step_origin.value==True:
        enrich_step_origin.icon = 'check'
    elif enrich_step_origin.value==False:
        enrich_step_origin.icon = ''
        
@enrich_step_isochrone.observe
def step_isochrone_check(*ignore):
    if enrich_step_isochrone.value==True:
        enrich_step_isochrone.icon = 'check'
    elif enrich_step_isochrone.value==False:
        enrich_step_isochrone.icon = ''

        
############################################################################
origin_tablename = widgets.Text(
                    placeholder='Table Name',
                    description='Table Name:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))

origin_var = widgets.Text(value='lat,lng',
                    placeholder='Y, lat, lng, etc ... (e.g. rf,lat,lng )',
                    description='Variable:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))
############################################################################

iso_mode = widgets.Dropdown(options=['car', 'walk'],
                            disabled=False,
                            value='car',
                            description='Mode: ',
                            layout=widgets.Layout(height='auto', width='30%'))

iso_ranges = widgets.Text(value='',
                          placeholder='Travel Time Values in Seconds (e.g. 60,180,240, ...)',
                          description='Ranges: ',
                          layout=widgets.Layout(height='auto', width='60%'),
                          disabled=False)
############################################################################

default_provider = 'usa_acs'
default_datasets = list(Catalog().provider(provider_id=default_provider).datasets.to_dataframe()['id'].values)

enrich_feature_providers = widgets.Tab(layout=widgets.Layout(width='250px', height='250px'))
enrich_feature_providers.set_title(0, 'Provider')
enrich_feature_providers.children = [widgets.Select(rows=10, 
                                                    value=default_provider,
                                                    options=['usa_acs', 'ags'], 
                                                    layout=widgets.Layout(width='100%'))] 

enrich_feature_datasets = widgets.Tab(layout=widgets.Layout(width='300px', height='250px'))
enrich_feature_datasets.set_title(0, 'Dataset')
enrich_feature_datasets.children = [widgets.Select(rows=10, 
                                                   value='',
                                                   options=default_datasets+[''],
                                                   layout=widgets.Layout(width='100%'))] 

enrich_feature_features = widgets.Tab(layout=widgets.Layout(width='300px', height='250px'))
enrich_feature_features.set_title(0, 'Feature')
enrich_feature_features.children = [widgets.SelectMultiple(rows=10, layout=widgets.Layout(width='100%'))] 
enrich_feature_features.children[0].options = []

enrich_feature_selected = widgets.Tab(layout=widgets.Layout(width='300px', height='250px'))
enrich_feature_selected.set_title(0, 'Selected [0]')
enrich_feature_selected.children = [widgets.SelectMultiple(rows=10, layout=widgets.Layout(width='100%'))] 
enrich_feature_selected.children[0].options = []

def dataset_update(*ignore):
    provider = enrich_feature_providers.children[0].value
    enrich_feature_datasets.children[0].options = list(Catalog().provider(provider_id=provider).datasets.to_dataframe()['id'].values) + ['']
    enrich_feature_datasets.children[0].value = ''

def feature_update(*ignore):
    if enrich_feature_datasets.children[0].value != '':
        dataset = Dataset.get(enrich_feature_datasets.children[0].value)
        enrich_feature_features.children[0].options = dataset.variables.to_dataframe()['id'].values
    else:
        enrich_feature_features.children[0].options = []

enrich_feature_providers.children[0].observe(dataset_update, ['value', 'options'])
enrich_feature_datasets.children[0].observe(feature_update, ['value', 'options'])


add_feature = widgets.Button(description="+", layout=widgets.Layout(width='80%', justify_content='center'))
remove_feature = widgets.Button(description="-", layout=widgets.Layout(width='80%', justify_content='center'))

@add_feature.on_click
def add_selected_feature(*ignore):
    if len(enrich_feature_features.children[0].value) != 0:
        enrich_feature_selected.children[0].options = list(set(list(enrich_feature_selected.children[0].options) + \
                                                               list(enrich_feature_features.children[0].value)))
        enrich_feature_selected.set_title(0, f'Selected [{len(enrich_feature_selected.children[0].options)}]')
        
@remove_feature.on_click
def remove_selected_feature(*ignore):
    if len(enrich_feature_selected.children[0].value) != 0:
        all_selected = list(enrich_feature_selected.children[0].options)
        for _ in enrich_feature_selected.children[0].value:
            all_selected.remove(_)
        enrich_feature_selected.children[0].options = all_selected
        enrich_feature_selected.set_title(0, f'Selected [{len(enrich_feature_selected.children[0].options)}]')
        

feature_info_board = widgets.HTML(disabled=True,
                                  layout=widgets.Layout(width='99%', 
                                                        height='auto',
                                                        display='flex',
                                                        flex_flow='row',
                                                        justify_content='center'))

def update_feature_info(*ignore):
    if enrich_feature_datasets.children[0].value != '':
        feature_info_board.value = f"<b>[Dataset]</b>: {Dataset.get(enrich_feature_datasets.children[0].value).to_dict()['name']}"
        feature_info_board.value += '<br/>'
        if len(enrich_feature_features.children[0].value) != 0:
            feature_info_board.value += f"<b>[Feature]</b>: {Variable.get(enrich_feature_features.children[0].value[0]).to_dict()['description']}"
            feature_info_board.value += '<br/>'
    else:
        feature_info_board.value = ''


enrich_feature_providers.children[0].observe(update_feature_info, ['value', 'options'])
enrich_feature_datasets.children[0].observe(update_feature_info, ['value', 'options'])
enrich_feature_features.children[0].observe(update_feature_info, ['value', 'options'])

############################################################################

enrich_accordion = widgets.Accordion(children=[widgets.HBox([origin_tablename, origin_var]),
                                               widgets.HBox([iso_mode, iso_ranges]),
                                               widgets.Text(value='tmp_iso',
                                                            placeholder='Table Name',
                                                            description='Table Name:',
                                                            disabled=False,
                                                            layout=widgets.Layout(width='50%')),
                                               widgets.VBox([widgets.HBox([enrich_feature_providers, 
                                                                           enrich_feature_datasets, 
                                                                           enrich_feature_features, 
                                                                           widgets.VBox([add_feature, remove_feature],
                                                                                         layout=widgets.Layout(width='50px', 
                                                                                                               display='flex',
                                                                                                               height='auto', 
                                                                                                               justify_content='center')),
                                                                           enrich_feature_selected]),
                                                             feature_info_board]),
                                               widgets.VBox([widgets.Text(value='tmp_origin',
                                                                         placeholder='Table Name',
                                                                         description='Origin:',
                                                                         disabled=False,
                                                                         layout=widgets.Layout(display='flex', width='auto'))
                                                            ])
                                              ])

enrich_accordion.set_title(0, 'Tables')
enrich_accordion.set_title(1, 'Isochrone Setting')
enrich_accordion.set_title(2, 'Isochrone Save')
enrich_accordion.set_title(3, 'Enrichment Features')
enrich_accordion.set_title(4, 'Enrichment Save')


@enrich_step_run.on_click
def run_enrichment(*ignore):
    if (enrich_step_origin.value == True) and (enrich_accordion.children[-1].children[0].value != ''):
        table_name = origin_tablename.value
        locations = CartoDataFrame.from_carto(table_name)
    
        if (enrich_step_isochrone.value == True) and (iso_ranges.value != '') and (enrich_accordion.children[2].value != ''):
            mode = iso_mode.value
            ranges = [float(_.strip()) for _ in enrich_accordion.children[1].children[1].value.split(',')]
            if len(ranges) == 1:
                locations_isochrones, _ = Isolines().isochrones(locations, ranges=ranges, mode=mode)
                locations_isochrones.to_carto(table_name=enrich_accordion.children[2].value, if_exists='replace')

        features = list(enrich_feature_selected.children[0].options)
        if len(features) != 0:
            origin = locations.copy()
            origin['geometry'] = origin['the_geom']
            if enrich_step_isochrone.value == True:
                iso = CartoDataFrame.from_carto(enrich_accordion.children[2].value)
                origin['geometry'] = iso['the_geom']

            enrichment_origin = origin[[_ for _ in ['geometry']+[_.strip() for _ in origin_var.value.split(',') if _!='']]]
            enrichment_origin = Enrichment().enrich_polygons(enrichment_origin, features)
            enrichment_origin.to_carto(table_name=enrich_accordion.children[-1].children[0].value, if_exists='replace')
        
################################################################################################################################
train_tablename = widgets.Text(
                    placeholder='Training Data Tabel Name',
                    description='Tabel Name:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))
train_tablename_link = widgets.jslink((enrich_accordion.children[-1].children[0], 'value'), (train_tablename, 'value'))

train_latlng = widgets.Text(
                    value='latitude,longitude',
                    placeholder='Training Data Lat/Lng Columns (e.g. lat,lng)',
                    description='Lat/Lng:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))
train_id_col = widgets.Text(
                    value='id',
                    placeholder='Training Data ID Columns',
                    description='ID:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))
train_target_col = widgets.Text(
                    value='sales_yearly',
                    placeholder='Training Data Target Variable',
                    description='Y:',
                    disabled=False,
                    layout=widgets.Layout(width='50%'))

train_features = widgets.Tab(layout=widgets.Layout(width='50%', height='250px'))
train_features.set_title(0, 'Feature')
train_features.children = [widgets.SelectMultiple(rows=10, layout=widgets.Layout(width='100%'))] 
train_features.children[0].options = []

train_features_selected = widgets.Tab(layout=widgets.Layout(width='50%', height='250px'))
train_features_selected.set_title(0, 'Selected [0]')
train_features_selected.children = [widgets.SelectMultiple(rows=10, layout=widgets.Layout(width='100%'))] 
train_features_selected.children[0].options = []

train_feature_search = widgets.Button(
    value=True,
    description='Search',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

@train_feature_search.on_click
def train_available_feature(*ignore):
    train_data = CartoDataFrame.from_carto(train_tablename.value)
    used_features = [train_target_col.value, train_id_col.value] + train_latlng.value.split(',') + ['geometry', 'the_geom']
    train_features.children[0].options = [_ for _ in list(set(train_data.columns)) if not _ in used_features]
    train_features_selected.children[0].options = []

    
train_add_feature = widgets.Button(description="+", layout=widgets.Layout(width='80%', justify_content='center'))
train_remove_feature = widgets.Button(description="-", layout=widgets.Layout(width='80%', justify_content='center'))

@train_add_feature.on_click
def train_add_selected_feature(*ignore):
    if len(train_features.children[0].value) != 0:
        train_features_selected.children[0].options = list(set(list(train_features_selected.children[0].options) + \
                                                               list(train_features.children[0].value)))
        train_features_selected.set_title(0, f'Selected [{len(train_features_selected.children[0].options)}]')
        
@train_remove_feature.on_click
def train_remove_selected_feature(*ignore):
    if len(train_features_selected.children[0].value) != 0:
        all_selected = list(train_features_selected.children[0].options)
        for _ in train_features_selected.children[0].value:
            all_selected.remove(_)
        train_features_selected.children[0].options = all_selected
        train_features_selected.set_title(0, f'Selected [{len(train_features_selected.children[0].options)}]')
            

predict_api_endpoint = widgets.Text(
                        value='http://0f521fda.ngrok.io/',
                        placeholder='http://*!#$!#$.ngrok.io/',
                        description='API: ',
                        disabled=False,
                        layout=widgets.Layout(width='50%'))
model_name = widgets.Text(
                        placeholder='Model Name',
                        description='Model: ',
                        disabled=False,
                        layout=widgets.Layout(width='50%'))

training_run = widgets.Button(
    value=True,
    description='Run',
    disabled=False,
    button_style='success',
    tooltip='Description',
    icon='check'
)

@training_run.on_click
def training_run_run(*ignore):
    if (train_tablename.value != '') and (train_id_col.value != '') and (train_latlng.value != '') and (len(train_features_selected.children[0].options)!=0) and (train_target_col.value != '') and (model_name.value != ''):        
        if (predict_api_endpoint.value != ''):
            call = f"""white_space_training?api_key={get_default_credentials().api_key}\
            &username={get_default_credentials().username}\
            &training_table={train_tablename.value}\
            &id_colname={train_id_col.value}\
            &lnglat_colname={train_latlng.value.strip()}\
            &feature_colname={','.join(list(train_features_selected.children[0].options))}\
            &target_colname={train_target_col.value}\
            &model_name={model_name.value}\
            &random_seed=112358"""
            call = call.replace(' ', '')
            print(call)
            run_api(ngrok_address=predict_api_endpoint.value, api_call=call, pause=1, timeout=600)
################################################################################################################################
refresh_model_list = widgets.Button(
    value=True,
    description='Refresh',
    disabled=False,
    button_style='info',
    icon='circle'
)

model_board = widgets.Tab(layout=widgets.Layout(width='250px', height='250px'))
model_board.set_title(0, 'Model Name')
model_board.children = [widgets.Select(rows=10, 
                                       value='',
                                       options=[''], 
                                       layout=widgets.Layout(width='100%'))] 

model_info = widgets.Tab(layout=widgets.Layout(width='900px', height='320px'))
model_info.set_title(0, 'Model Info')
model_info.children = [widgets.HTML(value='', layout=widgets.Layout(width='100%', height='100%'))] 

@refresh_model_list.on_click
def refresh_model_list_run(*ignore):
    import yaml  
    if predict_api_endpoint.value != '':
        call = "white_space_list_model"
        all_models = run_api(ngrok_address=predict_api_endpoint.value, api_call=call, pause=1, timeout=600)
        print(all_models.text)
        model_board.children[0].options = yaml.load(all_models.text) + ['']    
        model_board.children[0].value = ''
        model_info.children[0].value = ''
        
def refresh_model_info(*ignore):
    import yaml
    if model_board.children[0].value != '':
        call = f"white_space_check_model?model_name={model_board.children[0].value}"
        info = run_api(ngrok_address=predict_api_endpoint.value, api_call=call, pause=1, timeout=600)
        info = yaml.load(info.text)
        model_info.children[0].value = "<b>[Features]</b>:<br/>" + str(info[0]) + '<br/>' 
        model_info.children[0].value += "<b>[Hyperparameters (RandomForest)]</b>:<br/>" + str(info[2]) + '<br/>' 
        model_info.children[0].value += "<b>[Hyperparameters (Regression Kriging)]</b>:<br/>" + str(info[1]) + '<br/>'
        
model_board.children[0].observe(refresh_model_info, ['value', 'options'])
################################################################################################################################
test_tablename = widgets.Text(
                    value='ui_enrich',
                    placeholder='Test Data Tabel Name',
                    description='Tabel Name:',
                    disabled=False,
                    layout=widgets.Layout(width='70%'))

test_latlng = widgets.Text(
                    value='latitude,longitude',
                    placeholder='Test Data Lat/Lng Columns (e.g. lat,lng)',
                    description='Lat/Lng:',
                    disabled=False,
                    layout=widgets.Layout(width='70%'))
test_id_col = widgets.Text(
                    value='id',
                    placeholder='Test Data ID Columns',
                    description='ID:',
                    disabled=False,
                    layout=widgets.Layout(width='70%'))

predict_model_name = widgets.Text(
                    placeholder='Model Name For Prediction',
                    description='Model: ',
                    disabled=False,
                    layout=widgets.Layout(width='40%'))

predict_feature_search = widgets.Button(
    value=True,
    description='Search',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

predict_feature_validate = widgets.Button(
    value=True,
    description='Validate',
    disabled=True,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Validate if the test data contains all selected columns of the model?',
    icon='U+02620'
)

predict_feature_list = widgets.Tab(layout=widgets.Layout(width='70%', height='250px', justify_content='center'))
predict_feature_list.set_title(0, 'Selected [0]')
predict_feature_list.children = [widgets.SelectMultiple(rows=10, layout=widgets.Layout(width='100%', flex_flow='row', justify_content='center'))] 
predict_feature_list.children[0].options = []
predict_feature_list.children[0].disabled=True

test_output_tablename = widgets.Text(
                    value='ui_enrich_output',
                    placeholder='Test Output Tabel Name',
                    description='Output: ',
                    disabled=False,
                    layout=widgets.Layout(width='70%'))

predict_button = widgets.Button(
    value=True,
    description='Predict',
    disabled=True,
    button_style='primary',
    icon=''
)

predict_preview_button = widgets.Button(
    value=True,
    description='Preview',
    disabled=False,
    button_style='warning',
    icon=''
)

predict_preview_map = widgets.HTML()

@predict_feature_search.on_click
def predict_feature_search_run(*ignore):
    import yaml
    yaml.warnings({'YAMLLoadWarning': False})
    if predict_model_name.value != '':
        call = f"white_space_check_model?model_name={predict_model_name.value}"
        all_models = run_api(ngrok_address=predict_api_endpoint.value, api_call=call, pause=1, timeout=600)
        print(yaml.safe_load(all_models.text)[0])
        predict_feature_list.children[0].options = yaml.safe_load(all_models.text)[0]
        predict_feature_list.set_title(0, f'Selected [{len(yaml.safe_load(all_models.text)[0])}]')
        predict_feature_validate.disabled=False
        predict_feature_validate.style.button_color='#C0C0C0'
        predict_feature_validate.icon = ''
        predict_button.disabled=True
        
        
@predict_feature_validate.on_click
def predict_feature_validate_run(*ignore):
    if (predict_model_name.value != '') and (test_tablename.value != ''):
        test_all_cols = CartoDataFrame.from_carto(test_tablename.value).columns
        if len(set(predict_feature_list.children[0].options).difference(set(test_all_cols))) == 0:
            predict_feature_validate.icon = 'check'
            predict_feature_validate.style.button_color = 'lightgreen'
            predict_button.disabled=False
        else:
            predict_feature_validate.icon = 'U+02620'
            predict_feature_validate.style.button_color = 'red'
            
@predict_button.on_click
def predict_button_run(*ignore):
    call = f"""white_space_predict?api_key={get_default_credentials().api_key}\
                &username={get_default_credentials().username}\
                &test_table={test_tablename.value}\
                &output_table={test_output_tablename.value}\
                &id_colname={test_id_col.value}\
                &lnglat_colname={test_latlng.value.strip()}\
                &model_name={predict_model_name.value}\
                &random_seed=112358"""
    call = call.replace(' ', '')
    print(call)
    global_moran_i = run_api(ngrok_address=predict_api_endpoint.value, api_call=call, pause=1, timeout=600)
    print(global_moran_i.text)
    
@predict_preview_button.on_click
def predict_preview_button_run(*ignore):
    from sklearn.preprocessing import MinMaxScaler
    from shapely.geometry import Point
    if test_output_tablename.value != '':
        res= CartoDataFrame.from_carto(test_output_tablename.value)
        minmax = MinMaxScaler(feature_range=(20,50))
        res['prediction_width'] = minmax.fit_transform(res['rfrk'].values.reshape(-1, 1))
        res['upper_width'] = 3 + res['prediction_width'] * res['upper'] / res['rfrk']
        res['lower_width'] = -3 + res['prediction_width'] * res['lower'] / res['rfrk']

        res_map = Map(layers=[Layer(res, style={'color':' transparent', 'strokeColor': 'opacity(yellow, 0.75)', 'strokeWidth': 1}),
                        Layer(res.drop(columns=['the_geom']).assign(geometry=res.apply(lambda x: Point(x['lng'], x['lat']), axis=1)), 
                              style={'color': 'red', 
                                     'symbol': 'house',
                                     'strokeColor': 'transparent', 
                                     'width':5}),
                        Layer(res.drop(columns=['the_geom']).assign(geometry=res.apply(lambda x: Point(x['lng'], x['lat']), axis=1)), 
                              style={'color': 'transparent', 
                                     'strokeColor': '#9EB9F3',
                                     'strokeWidth': '1.5',
                                     'width': '$upper_width'},
                              legend={'type': 'color-category-point', 'title':'80% Percentile Prediction'}),
                        Layer(res.drop(columns=['the_geom']).assign(geometry=res.apply(lambda x: Point(x['lng'], x['lat']), axis=1)), 
                              style={'color': 'transparent', 
                                     'strokeColor': '#66C5CC',
                                     'strokeWidth': '2.5',
                                     'width': '$prediction_width'},
                             legend={'type': 'color-category-point', 'title':'Regression Kriging Prediction'}),
                        Layer(res.drop(columns=['the_geom']).assign(geometry=res.apply(lambda x: Point(x['lng'], x['lat']), axis=1)), 
                              style={'color': 'transparent', 
                                     'strokeColor': '#D3B484',
                                     'strokeWidth': '1.5',
                                     'width': '$lower_width'},
                             legend={'type': 'color-category-point', 'title':'20% Percentile Prediction'}),
                       ], 
                basemap=basemaps.darkmatter)
        predict_preview_map.value = res_map._repr_html_()

################################################################################################################################

accordion = widgets.Accordion(children=[widgets.VBox([auth_username, widgets.HBox([auth_apikey, widgets.HTML('<span style="padding-left:100px"></span>'), auth_confirm])]),
                                        widgets.VBox([widgets.HBox([data_tablename, data_preview, data_confirm]),
                                                      data_preview_map]),
                                        widgets.VBox([widgets.HBox([enrich_step_origin, arrow,
                                                                    enrich_step_isochrone, arrow,
                                                                    enrich_step_run], 
                                                      layout=widgets.Layout(height='auto', 
                                                                            flex_flow='row',
                                                                            justify_content='center')),
                                                      enrich_accordion
                                                     ]),
                                        widgets.VBox([widgets.VBox([widgets.HBox([train_tablename, 
                                                                                  widgets.HTML('<span style="padding-left:100px"></span>'), 
                                                                                  train_feature_search]),
                                                                    train_id_col, train_latlng, train_target_col]),
                                                      widgets.HTML('<br/>'),
                                                      widgets.HBox([train_features,
                                                                    widgets.VBox([train_add_feature, train_remove_feature],
                                                                                 layout=widgets.Layout(width='50px', 
                                                                                                       display='flex',
                                                                                                       height='auto',
                                                                                                       justify_content='center')),
                                                                    train_features_selected]),
                                                      widgets.HTML('<br/>'),
                                                      widgets.HBox([predict_api_endpoint, 
                                                                    model_name, 
                                                                    widgets.HTML('<span style="padding-left:100px"></span>'), 
                                                                    training_run])
                                                     ]),
                                        widgets.HBox([widgets.VBox([refresh_model_list, widgets.HTML('<br/>'), model_board]),
                                                      widgets.HTML('<span style="padding-left:100px"></span>'), 
                                                      widgets.VBox([model_info])]),
                                        
                                        widgets.VBox([test_tablename, 
                                                      test_id_col, 
                                                      test_latlng,
                                                      test_output_tablename, 
                                                      widgets.HTML('<br/>'),
                                                      widgets.VBox([widgets.HBox([predict_model_name,
                                                                                  widgets.HTML('<span style="padding-left:100px"></span>'),
                                                                                  predict_feature_search,
                                                                                  widgets.HTML('<span style="padding-left:10px"></span>'),
                                                                                  predict_feature_validate])]),
                                                      widgets.HTML('<br/>'),
                                                      widgets.HBox([widgets.HTML('<span style="padding-left:30px"></span>'),
                                                                    predict_feature_list], 
                                                                   layout=widgets.Layout(justify_content='unset')),
                                                      widgets.HTML('<br/>'),
                                                      widgets.HBox([predict_api_endpoint, 
                                                                    widgets.HTML('<span style="padding-left:50px"></span>'),
                                                                    predict_button, 
                                                                    widgets.HTML('<span style="padding-left:20px"></span>'),
                                                                    predict_preview_button]),
                                                      predict_preview_map
                                                     ])
                                       ])
accordion.set_title(0, 'Carto Account')
accordion.set_title(1, 'Data')
accordion.set_title(2, 'Enrichment')
accordion.set_title(3, 'Training')
accordion.set_title(4, 'Model')
accordion.set_title(5, 'Predict')
accordion